In [2]:
import pandas as pd
import re
from langdetect import detect


books_df = pd.read_csv('dataset/books_data_with_id.csv')




In [12]:
books_data_df = books_df.sample(n=10000, random_state=1)

print("Sample DataFrame of 5000 books:")
print(books_data_df)



Sample DataFrame of 5000 books:
        book_id                                              Title  \
29329     29330                          When Egypt Ruled the East   
18075     18076                                         Murder One   
190574   190575  Growing Up Plain Among Conservative Wilburite ...   
97879     97880                Handbook of Inter-Rater Reliability   
89506     89507  3 Sides of You: Unlocking the Way You Think, W...   
...         ...                                                ...   
91003     91004                                        The Turning   
186951   186952                              The Sweetheart Season   
39996     39997      Rain and Hail (Let's-Read-and-Find-Out Books)   
118613   118614  John L. Stoddard's lectures; Supplementary volume   
100810   100811                       The church reclaims the city   

                                              description  \
29329   Here, adequately presented for the first time ...   
18075

In [13]:
books_rating_with_id_df = pd.read_csv('dataset/Books_rating_with_id.csv')

books_rating_df = books_rating_with_id_df[books_rating_with_id_df['Title'].isin(books_data_df['Title'])]

books_rating_df = books_rating_df[['User_id', 'review/score', 'book_id', 'Title']].dropna(subset=['User_id', 'review/score', 'book_id', 'Title'])

print("Df mới của book rating:")
print(books_rating_df)

Df mới của book rating:
                User_id  review/score  book_id  \
74       A1CHPTJILEAL74           5.0       14   
76       A30DX2BO4Y4NLU           3.0       14   
77        A29Z0B2L367ZO           5.0       14   
175       AF2VPO1D4ZJ0X           5.0       29   
304      A1TIZI060W4BD9           5.0       54   
...                 ...           ...      ...   
2999776   AHWRWEV4MJQ5T           5.0   132376   
2999786  A14OJS0VWMOSWO           5.0   132379   
2999787  A3BH49ZKESHDID           5.0   132379   
2999788  A3U2RZFOQ29U1Z           4.0   132379   
2999888  A1DRCA6UNGP19D           5.0   132386   

                                                     Title  
74       Vector Quantization and Signal Compression (Th...  
76       Vector Quantization and Signal Compression (Th...  
77       Vector Quantization and Signal Compression (Th...  
175                    Ricky Williams: Dreadlocks to Ditka  
304                                                 Seance  
...      

In [14]:
import pandas as pd
# Đếm số lần đánh giá cho mỗi User_id
user_rating_counts = books_rating_df.groupby('User_id').size().reset_index(name='Rated_Books_Count')

# Lọc các User_id đã đánh giá ít nhất 10 cuốn sách
users_with_at_least_10_ratings = user_rating_counts[user_rating_counts['Rated_Books_Count'] >= 10]['User_id']

# Lọc các hàng trong 'books_rating_df' với User_id thỏa mãn điều kiện
filtered_books_rating_df = books_rating_df[books_rating_df['User_id'].isin(users_with_at_least_10_ratings)]

print("Df mới của book rating sau khi loại bỏ các User_id đánh giá:")
print(filtered_books_rating_df)



Df mới của book rating sau khi loại bỏ các User_id đánh giá:
                User_id  review/score  book_id  \
304      A1TIZI060W4BD9           5.0       54   
307      A1OB5L3WMHJAD9           4.0       54   
371       ATDE9JYCPI0L1           5.0       67   
459       A4FX5YCJA630V           3.0       74   
1739     A1YB29BIDULEYE           5.0      153   
...                 ...           ...      ...   
2997883  A2UIXU97JYCPZG           3.0   132323   
2997886  A1OBNY1WB38LMI           4.0   132323   
2998172  A3AUL23GMCOP2A           3.0   132362   
2999786  A14OJS0VWMOSWO           5.0   132379   
2999787  A3BH49ZKESHDID           5.0   132379   

                                                 Title  
304                                             Seance  
307                                             Seance  
371                 Hidden History of the Kovno Ghetto  
459                 Night World: Daughters Of Darkness  
1739                      The Scarlet Letter A Romanc

In [15]:
books_rating_df = filtered_books_rating_df

In [16]:
from sklearn.model_selection import train_test_split
grouped = books_rating_df.groupby('book_id')


train_list = []
test_list = []

for book_id, group in grouped:
    if len(group) < 2:
        train_list.append(group)
    else:
        train_group, test_group = train_test_split(group, test_size=0.5, random_state=42)
        train_list.append(train_group)
        test_list.append(test_group)

train_data = pd.concat(train_list)
test_data = pd.concat(test_list)


print("Tập train:")
print(train_data)

print("\nTập test:")
print(test_data)

Tập train:
                User_id  review/score  book_id  \
304      A1TIZI060W4BD9           5.0       54   
371       ATDE9JYCPI0L1           5.0       67   
459       A4FX5YCJA630V           3.0       74   
2770431  A3M174IC0VXOS2           5.0      142   
1918413  A2F079BGQTQ3BM           4.0      153   
...                 ...           ...      ...   
2817391  A14OJS0VWMOSWO           5.0   212132   
2819403  A3MV1KKHX51FYT           3.0   212198   
2819450  A319KYEIAZ3SON           4.0   212198   
2819543  A14OJS0VWMOSWO           5.0   212206   
2820968  A2OJW07GQRNJUT           5.0   212308   

                                                     Title  
304                                                 Seance  
371                     Hidden History of the Kovno Ghetto  
459                     Night World: Daughters Of Darkness  
2770431                                            Gorilla  
1918413                       The Scarlet Letter A Romance  
...                   

In [17]:
common_users = set(train_data['User_id']).intersection(set(test_data['User_id']))

# Tạo danh sách để lưu số liệu thống kê
statistics = []

# Kiểm tra và đếm số lượng đánh giá của từng user trong cả hai tập
for user in common_users:
    train_ratings_count = train_data[train_data['User_id'] == user].shape[0]
    test_ratings_count = test_data[test_data['User_id'] == user].shape[0]
    
    # Lưu thông tin vào danh sách
    statistics.append({
        'User_id': user,
        'Train_Ratings_Count': train_ratings_count,
        'Test_Ratings_Count': test_ratings_count
    })

# Tạo DataFrame từ danh sách số liệu thống kê
stats_df = pd.DataFrame(statistics)

# Hiển thị bảng thống kê
print(stats_df)

            User_id  Train_Ratings_Count  Test_Ratings_Count
0     A8IZFSUJG5IV0                    6                   4
1    A1PKKR5X3HZ8MR                    3                   7
2    A2VZ11U5DXM8J5                   10                  14
3     AMFA6FJHTMU8U                    9                   3
4     AUEY946M1L939                   10                   5
..              ...                  ...                 ...
911  A3ICDLUQ3V2QY2                    6                   4
912  A2VKWLCNZF4ZVB                   24                  10
913  A2ZSC81MXLBELX                    5                   9
914   A1RRFA2LEWUU9                    7                   3
915   ANEDXRFDZDL18                    6                  10

[916 rows x 3 columns]


In [18]:
train_book_ids = set(train_data['book_id'].unique())
test_book_ids = set(test_data['book_id'].unique())

# Tạo danh sách book để sử dụng
all_book_ids = train_book_ids.union(test_book_ids)

print(all_book_ids)
print(len(all_book_ids))


{114691, 155654, 204807, 155660, 188432, 57364, 16405, 163863, 188442, 163868, 57373, 57375, 155683, 106546, 54, 32824, 106555, 49213, 155709, 67, 204872, 74, 65611, 139340, 139342, 81999, 172110, 57428, 188502, 122967, 65626, 122971, 196699, 8287, 32878, 82031, 180339, 131189, 123001, 188538, 131197, 155780, 8331, 142, 65681, 172182, 139415, 153, 155802, 188569, 188575, 90272, 188576, 164010, 188588, 180397, 8366, 41135, 65716, 106678, 8379, 194, 98500, 180424, 131275, 8400, 57555, 180436, 164058, 90331, 98530, 155876, 147688, 172265, 235, 164076, 41198, 164087, 98553, 90362, 41211, 131329, 139522, 172290, 196875, 205081, 33052, 33053, 164129, 164131, 123172, 172323, 74026, 123180, 172335, 106800, 139571, 49465, 90426, 16699, 155964, 205114, 8512, 115008, 41290, 172362, 16719, 172373, 196950, 147799, 82264, 65881, 188759, 65883, 115040, 205154, 205160, 180589, 65903, 57713, 90483, 115060, 90485, 16758, 196983, 8569, 49530, 196987, 188797, 131455, 41345, 205194, 8587, 8590, 106894, 741

In [19]:
books_rating_df = train_data

In [20]:
books_rating_df['review/score'] = pd.to_numeric(books_rating_df['review/score'], errors='coerce')
books_rating_df = books_rating_df[books_rating_df['User_id'].astype(str).str.strip() != '']
books_rating_df = books_rating_df[books_rating_df['review/score'].astype(str).str.strip() != '']

books_rating_df = books_rating_df.groupby(['User_id', 'book_id'], as_index=False)['review/score'].mean()


In [46]:
import pandas as pd

def create_user_item_matrix(books_rating_df):
    user_item_matrix = books_rating_df.pivot(index='book_id', columns='User_id', values='review/score')

    return user_item_matrix

user_item_matrix = create_user_item_matrix(books_rating_df)

print("Ma trận đánh giá ban đầu:")
print(user_item_matrix)


Ma trận đánh giá ban đầu:
User_id  A104BTD1MMNQVF  A106016KSI0YQ  A1065304SY1HF8  A106C5BHWK9SKP  \
book_id                                                                  
54                  NaN            NaN             NaN             NaN   
67                  NaN            NaN             NaN             NaN   
74                  NaN            NaN             NaN             NaN   
142                 NaN            NaN             NaN             NaN   
153                 NaN            NaN             NaN             NaN   
...                 ...            ...             ...             ...   
212076              NaN            NaN             NaN             NaN   
212132              NaN            NaN             NaN             NaN   
212198              NaN            NaN             NaN             NaN   
212206              NaN            NaN             NaN             NaN   
212308              NaN            NaN             NaN             NaN   

User_id  A1

In [ ]:
import pandas as pd

def create_user_item_matrix(books_rating_df):
    user_item_matrix = books_rating_df.pivot(index='book_id', columns='User_id', values='review/score')

    return user_item_matrix

user_item_matrix = create_user_item_matrix(books_rating_df)

print("Ma trận đánh giá ban đầu:")
print(user_item_matrix)


Ma trận đánh giá ban đầu:
User_id  A104BTD1MMNQVF  A106016KSI0YQ  A1065304SY1HF8  A106C5BHWK9SKP  \
book_id                                                                  
54                  NaN            NaN             NaN             NaN   
67                  NaN            NaN             NaN             NaN   
74                  NaN            NaN             NaN             NaN   
142                 NaN            NaN             NaN             NaN   
153                 NaN            NaN             NaN             NaN   
...                 ...            ...             ...             ...   
212076              NaN            NaN             NaN             NaN   
212132              NaN            NaN             NaN             NaN   
212198              NaN            NaN             NaN             NaN   
212206              NaN            NaN             NaN             NaN   
212308              NaN            NaN             NaN             NaN   

User_id  A1

In [69]:
import pandas as pd

def normalize_rating_matrix_by_item(user_item_matrix):
    # Tính trung bình các giá trị không phải NaN của từng item (hàng)
    item_mean = user_item_matrix.mean(axis=1, skipna=True)
    
    # Trừ giá trị trung bình khỏi các giá trị không phải NaN
    normalized_user_item_matrix = user_item_matrix.sub(item_mean, axis=0)
    
    return normalized_user_item_matrix


# normalized_user_item_matrix = normalize_rating_matrix_by_item(user_item_matrix)
normalized_user_item_matrix = normalize_rating_matrix_by_item(user_item_matrix)
print("\nMa trận đánh giá đã chuẩn hóa theo item mean:")
print(normalized_user_item_matrix)



Ma trận đánh giá đã chuẩn hóa theo item mean:
User_id  A104BTD1MMNQVF  A106016KSI0YQ  A1065304SY1HF8  A106C5BHWK9SKP  \
book_id                                                                  
54                  NaN            NaN             NaN             NaN   
67                  NaN            NaN             NaN             NaN   
74                  NaN            NaN             NaN             NaN   
142                 NaN            NaN             NaN             NaN   
153                 NaN            NaN             NaN             NaN   
...                 ...            ...             ...             ...   
212076              NaN            NaN             NaN             NaN   
212132              NaN            NaN             NaN             NaN   
212198              NaN            NaN             NaN             NaN   
212206              NaN            NaN             NaN             NaN   
212308              NaN            NaN             NaN           

In [87]:
normalized_user_item_matrix.fillna(0, inplace=True)
print(normalized_user_item_matrix)


User_id  A104BTD1MMNQVF  A106016KSI0YQ  A1065304SY1HF8  A106C5BHWK9SKP  \
book_id                                                                  
54               0.0000         0.0000          0.0000          0.0000   
67               0.0000         0.0000          0.0000          0.0000   
74               0.0000         0.0000          0.0000          0.0000   
142              0.0000         0.0000          0.0000          0.0000   
153              0.0000         0.0000          0.0000          0.0000   
...                 ...            ...             ...             ...   
212076           0.0000         0.0000          0.0000          0.0000   
212132           0.0000         0.0000          0.0000          0.0000   
212198           0.0000         0.0000          0.0000          0.0000   
212206           0.0000         0.0000          0.0000          0.0000   
212308           0.0000         0.0000          0.0000          0.0000   

User_id  A106QCX4I544YJ  A10A1S5NAQBT

In [88]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
similarity_df = normalized_user_item_matrix.T.corr(method='pearson')
# similarity_df = pd.DataFrame(cosine_similarity(normalized_user_item_matrix.fillna(0)),
#                              index=normalized_user_item_matrix.index,  # Index là ID sách
#                              columns=normalized_user_item_matrix.T.index)  
# Hiển thị ma trận tương đồng
print("Ma trận tương đồng giữa các sách (items):")
print(similarity_df)


Ma trận tương đồng giữa các sách (items):
book_id  54      67      74      142     153     194     235     494     \
book_id                                                                   
54          NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
67          NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
74          NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
142         NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
153         NaN     NaN     NaN     NaN  1.0000     NaN     NaN     NaN   
...         ...     ...     ...     ...     ...     ...     ...     ...   
212076      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
212132      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
212198      NaN     NaN     NaN     NaN  0.0000     NaN     NaN     NaN   
212206      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
212308      NaN     NaN     NaN     NaN     NaN     NaN   

In [55]:
user_id = 'A14OJS0VWMOSWO'  # Ví dụ user_id

# Lấy các sách đã được người dùng đánh giá
user_ratings = user_item_matrix.loc[:, user_id]

# Kiểm tra các giá trị trong user_ratings
print("Giá trị đánh giá của người dùng:", user_id)
print(user_ratings)

# Kiểm tra thông tin chi tiết
print("\nThông tin mô tả về các giá trị:")
print(user_ratings.describe())

# Kiểm tra các giá trị không phải NaN
print("\nCác giá trị không phải NaN:")
print(user_ratings[user_ratings.notna()])

# Kiểm tra các giá trị lớn hơn 0
print("\nCác giá trị lớn hơn 0:")
print(user_ratings[user_ratings > 0])

Giá trị đánh giá của người dùng: A14OJS0VWMOSWO
book_id
54          NaN
67          NaN
74          NaN
142         NaN
153         NaN
          ...  
212076      NaN
212132   5.0000
212198      NaN
212206   5.0000
212308      NaN
Name: A14OJS0VWMOSWO, Length: 2362, dtype: float64

Thông tin mô tả về các giá trị:
count   213.0000
mean      5.0000
std       0.0000
min       5.0000
25%       5.0000
50%       5.0000
75%       5.0000
max       5.0000
Name: A14OJS0VWMOSWO, dtype: float64

Các giá trị không phải NaN:
book_id
1224     5.0000
2040     5.0000
2279     5.0000
2513     5.0000
2742     5.0000
          ...  
208228   5.0000
210141   5.0000
210836   5.0000
212132   5.0000
212206   5.0000
Name: A14OJS0VWMOSWO, Length: 213, dtype: float64

Các giá trị lớn hơn 0:
book_id
1224     5.0000
2040     5.0000
2279     5.0000
2513     5.0000
2742     5.0000
          ...  
208228   5.0000
210141   5.0000
210836   5.0000
212132   5.0000
212206   5.0000
Name: A14OJS0VWMOSWO, Length: 213, dtype

In [90]:
import pandas as pd
import numpy as np

def recommend_books(user_id, top_k=5):
    # Lấy các sách đã được người dùng đánh giá
    user_ratings = user_item_matrix.loc[:, user_id]
    normalized_user_ratings = normalized_user_item_matrix.loc[:, user_id]
    
    # Lấy các sách đã được đánh giá
    rated_books = user_ratings[user_ratings.notna()].index

    unrated_books = all_book_ids.difference(set(rated_books))
    # Kiểm tra nếu người dùng không đánh giá sách nào
    if rated_books.empty:
        print(f"Người dùng {user_id} chưa đánh giá sách nào.")
        return []

    # Khởi tạo một từ điển để lưu điểm gợi ý
    recommendation_scores = {}
    
    for unrated_book in unrated_books:
        # Kiểm tra nếu sách chưa được đánh giá không có trong ma trận tương đồng
        if unrated_book not in similarity_df.columns:
            continue
        
        # Lấy độ tương đồng của sách chưa được đánh giá với các sách đã được đánh giá
        similar_books = similarity_df[unrated_book].dropna()
        
        # Tính weighted_sum và total_similarity
        weighted_sum = 0
        total_similarity = 0
        
        for rated_book in rated_books:
            if rated_book in similar_books.index:
                weighted_sum += (similar_books[rated_book] * normalized_user_ratings[rated_book])
                total_similarity += abs(similar_books[rated_book])
        
        # Tránh chia cho 0
        if total_similarity != 0:
            recommendation_scores[unrated_book] = weighted_sum / total_similarity
    
    # Sắp xếp các sách gợi ý theo điểm và chọn n_recommendations
    recommended_books = sorted(recommendation_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
    
    # Kiểm tra nếu không có sách nào được gợi ý
    if not recommended_books:
        print(f"Không có sách nào để gợi ý cho người dùng {user_id}.")
        return []
    
    return recommended_books



In [ ]:
75    A14OJS0VWMOSWO                215
1747     AFVQZQ8PW0L                138
201   A1D2C0WDCSHUWZ                 89
564   A20EEWWSFMZ1PN                 72
516   A1X8VZWTOG8IS6

In [93]:
user_id = 'AFVQZQ8PW0L'  # Ví dụ user_id
top_k = 10
recommended_books = recommend_books(user_id, top_k)
print(recommended_books)

# Hiển thị kết quả gợi ý sách
if isinstance(recommended_books, str):
    print(recommended_books)
else:
    print("Gợi ý sách cho người dùng", user_id, ":")
    for book, score in recommended_books:
        print(f"Sách ID: {book}, Điểm gợi ý: {score:.4f}")


Không có sách nào có tổng độ tương đồng lớn hơn 0
[(164087, 1.1666666666666665), (148045, 1.1666666666666665), (17193, 1.1666666666666665), (157798, 1.1666666666666665), (19992, 1.1666666666666665), (178103, 1.1666666666666665), (31419, 1.1666666666666665), (179884, 1.1666666666666665), (57243, 1.1666666666666665), (142638, 0.9000000000000004)]
Gợi ý sách cho người dùng AFVQZQ8PW0L :
Sách ID: 164087, Điểm gợi ý: 1.1667
Sách ID: 148045, Điểm gợi ý: 1.1667
Sách ID: 17193, Điểm gợi ý: 1.1667
Sách ID: 157798, Điểm gợi ý: 1.1667
Sách ID: 19992, Điểm gợi ý: 1.1667
Sách ID: 178103, Điểm gợi ý: 1.1667
Sách ID: 31419, Điểm gợi ý: 1.1667
Sách ID: 179884, Điểm gợi ý: 1.1667
Sách ID: 57243, Điểm gợi ý: 1.1667
Sách ID: 142638, Điểm gợi ý: 0.9000


In [187]:
# book_id = 24

# book_info = books_data_df.loc[books_data_df['book_id'] == book_id]

# if not book_info.empty:
#     title = book_info['Title'].values[0]
#     description = book_info['description'].values[0]
#     categories = book_info['categories'].values[0]
#     author = book_info['authors'].values[0]
#     print(f"Thông tin cuốn sách với book_id {book_id}:")
#     print(f"Tiêu đề: '{title}'")
#     print(f"Mô tả: '{description}'")
#     print(f"Thể loại: {categories}")
#     print(f"Tác giả: {author}")
# else:
#     print(f"Không tìm thấy sách với book_id {book_id}.")


In [188]:
# book_id = 64109

# book_info = books_data_df.loc[books_data_df['book_id'] == book_id]

# if not book_info.empty:
#     title = book_info['Title'].values[0]
#     description = book_info['description'].values[0]
#     categories = book_info['categories'].values[0]
#     author = book_info['authors'].values[0]
#     print(f"Thông tin cuốn sách với book_id {book_id}:")
#     print(f"Tiêu đề: '{title}'")
#     print(f"Mô tả: '{description}'")
#     print(f"Thể loại: {categories}")
#     print(f"Tác giả: {author}")
# else:
#     print(f"Không tìm thấy sách với book_id {book_id}.")


In [105]:
import pandas as pd
import numpy as np

def recommend_books(user_id, top_k=5):
    # Lấy các sách đã được người dùng đánh giá
    user_ratings = user_item_matrix.loc[:, user_id]
    normalized_user_ratings = normalized_user_item_matrix.loc[:, user_id]
    
    # Lấy các sách đã được đánh giá
    rated_books = user_ratings[user_ratings.notna()].index

    unrated_books = all_book_ids.difference(set(rated_books))
    
    # Kiểm tra nếu người dùng không đánh giá sách nào
    if rated_books.empty:
        print(f"Người dùng {user_id} chưa đánh giá sách nào.")
        return []

    # Khởi tạo một từ điển để lưu điểm gợi ý
    recommendation_scores = {}
    no_similarity_data_books = []  # Các sách không có dữ liệu tương đồng
    total_similarity_zero_books = []  # Các sách có tổng độ tương đồng bằng 0
    no_rated_data_books = []
    for unrated_book in unrated_books:
        # Kiểm tra nếu sách chưa được đánh giá không có trong ma trận tương đồng
        if unrated_book not in similarity_df.columns:
            no_similarity_data_books.append(unrated_book)
            continue
        
        # Lấy độ tương đồng của sách chưa được đánh giá với các sách đã được đánh giá
        similar_books = similarity_df[unrated_book].dropna()
        
        # Tính weighted_sum và total_similarity
        weighted_sum = 0
        total_similarity = 0
        
        for rated_book in rated_books:
            if rated_book in similar_books.index:
                similarity = similar_books[rated_book]
                # Tính toán điểm gợi ý dù độ tương đồng có thể <= 0
                weighted_sum += (similarity * normalized_user_ratings[rated_book])
                total_similarity += abs(similarity)
            else:
                no_rated_data_books.append(rated_book)
        
        # Tránh chia cho 0
        if total_similarity > 0:
            recommendation_scores[unrated_book] = weighted_sum / total_similarity
        else:
            total_similarity_zero_books.append(unrated_book)
    
    # Sắp xếp các sách gợi ý theo điểm và chọn n_recommendations
    recommended_books = sorted(recommendation_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
    
    # Kiểm tra các trường hợp đặc biệt và đưa ra thông báo
    if no_similarity_data_books:
        print(f"Không có dữ liệu tương đồng cho các sách chưa được đánh giá")
    
    if total_similarity_zero_books:
        print(f"Không có sách nào có tổng độ tương đồng lớn hơn 0")
        
    if no_rated_data_books:
        print(f"Sách được đanh giá không có trong ma trận")
    
    # Kiểm tra nếu không có sách nào được gợi ý
    if not recommended_books:
        print(f"Không có sách nào để gợi ý cho người dùng {user_id}.")
        return []
    
    return recommended_books


In [106]:
import pandas as pd

def calculate_top_k_hits_and_metrics(user_id, k, train_data, test_data):
    recommended_books = recommend_books(user_id, k)
    book_list = [book_id for book_id, score in recommended_books]

    actual_books = test_data[test_data['User_id'] == user_id]['book_id'].tolist()

    hits = len(set(book_list) & set(actual_books))

    precision = hits / k if k > 0 else 0

    total_relevant_books = len(actual_books) 
    recall = hits / total_relevant_books if total_relevant_books > 0 else 0

    return hits, precision, recall

def evaluate_f1_score(k, train_data, test_data):
    user_ids = train_data['User_id'].unique() 
    total_hits = 0
    total_precision = 0
    total_recall = 0
    total_users = len(user_ids)

    for user_id in user_ids:
        hits, precision, recall = calculate_top_k_hits_and_metrics(user_id, k, train_data, test_data)
        total_hits += hits
        total_precision += precision
        total_recall += recall

    avg_precision = total_precision / total_users if total_users > 0 else 0
    avg_recall = total_recall / total_users if total_users > 0 else 0

    if avg_precision + avg_recall > 0:
        f1_score = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
    else:
        f1_score = 0

    return total_hits, avg_precision, avg_recall, f1_score

k = 10
total_hits, avg_precision, avg_recall, f1_score = evaluate_f1_score(k, train_data, test_data)

print(f"Tổng số hits: {total_hits}")
print(f"Precision trung bình: {avg_precision:.4f}")
print(f"Recall trung bình: {avg_recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")


Không có sách nào có tổng độ tương đồng lớn hơn 0
Sách được đanh giá không có trong ma trận
Không có sách nào có tổng độ tương đồng lớn hơn 0
Sách được đanh giá không có trong ma trận
Không có sách nào để gợi ý cho người dùng ATDE9JYCPI0L1.
Không có sách nào có tổng độ tương đồng lớn hơn 0
Sách được đanh giá không có trong ma trận
Không có sách nào có tổng độ tương đồng lớn hơn 0
Sách được đanh giá không có trong ma trận
Không có sách nào có tổng độ tương đồng lớn hơn 0
Sách được đanh giá không có trong ma trận
Không có sách nào có tổng độ tương đồng lớn hơn 0
Sách được đanh giá không có trong ma trận
Không có sách nào có tổng độ tương đồng lớn hơn 0
Sách được đanh giá không có trong ma trận
Không có sách nào có tổng độ tương đồng lớn hơn 0
Sách được đanh giá không có trong ma trận
Không có sách nào có tổng độ tương đồng lớn hơn 0
Sách được đanh giá không có trong ma trận
Không có sách nào có tổng độ tương đồng lớn hơn 0
Sách được đanh giá không có trong ma trận
Không có sách nào có tổ

In [29]:
import pandas as pd

def calculate_top_k_hits_and_metrics(user_id, k, train_data, test_data):
    recommended_books = recommend_books(user_id, k)
    book_list = [book_id for book_id, score in recommended_books]

    actual_books = test_data[test_data['User_id'] == user_id]['book_id'].tolist()

    hits = len(set(book_list) & set(actual_books))

    precision = hits / k if k > 0 else 0

    total_relevant_books = len(actual_books) 
    recall = hits / total_relevant_books if total_relevant_books > 0 else 0

    return hits, precision, recall

def evaluate_f1_score(k, train_data, test_data):
    user_ids = train_data['User_id'].unique() 
    total_hits = 0
    total_precision = 0
    total_recall = 0
    total_users = len(user_ids)

    for user_id in user_ids:
        hits, precision, recall = calculate_top_k_hits_and_metrics(user_id, k, train_data, test_data)
        total_hits += hits
        total_precision += precision
        total_recall += recall

    avg_precision = total_precision / total_users if total_users > 0 else 0
    avg_recall = total_recall / total_users if total_users > 0 else 0

    if avg_precision + avg_recall > 0:
        f1_score = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
    else:
        f1_score = 0

    return total_hits, avg_precision, avg_recall, f1_score

k = 20
total_hits, avg_precision, avg_recall, f1_score = evaluate_f1_score(k, train_data, test_data)

print(f"Tổng số hits: {total_hits}")
print(f"Precision trung bình: {avg_precision:.4f}")
print(f"Recall trung bình: {avg_recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")


Không có sách nào để gợi ý cho người dùng ATDE9JYCPI0L1.
Không có sách nào để gợi ý cho người dùng A36K2N527TXXJN.
Không có sách nào để gợi ý cho người dùng A33UDDOH82EROT.
Không có sách nào để gợi ý cho người dùng A3AUL23GMCOP2A.
Không có sách nào để gợi ý cho người dùng A2Z4KA3EFQWZOX.
Không có sách nào để gợi ý cho người dùng A3QBCX2QMKJ7AH.
Không có sách nào để gợi ý cho người dùng A3EX36SNRYD5VL.
Không có sách nào để gợi ý cho người dùng A1DTOHMM2Y5KY0.
Không có sách nào để gợi ý cho người dùng A324NNSMM9EFML.
Không có sách nào để gợi ý cho người dùng A1K9IG5MP8RW75.
Không có sách nào để gợi ý cho người dùng AU7ND6NOCX9IA.
Không có sách nào để gợi ý cho người dùng A1L5RDC8H9BB9S.
Không có sách nào để gợi ý cho người dùng A4H4KYSM2KQ85.
Không có sách nào để gợi ý cho người dùng A16CZRQL23NOIW.
Không có sách nào để gợi ý cho người dùng A2FS9CYCWC6HLW.
Không có sách nào để gợi ý cho người dùng AGV0RUQJHFW3S.
Không có sách nào để gợi ý cho người dùng A258HDMERSI6WJ.
Không có sách nào 

In [30]:
import pandas as pd

def calculate_top_k_hits_and_metrics(user_id, k, train_data, test_data):
    recommended_books = recommend_books(user_id, k)
    book_list = [book_id for book_id, score in recommended_books]

    actual_books = test_data[test_data['User_id'] == user_id]['book_id'].tolist()

    hits = len(set(book_list) & set(actual_books))

    precision = hits / k if k > 0 else 0

    total_relevant_books = len(actual_books) 
    recall = hits / total_relevant_books if total_relevant_books > 0 else 0

    return hits, precision, recall

def evaluate_f1_score(k, train_data, test_data):
    user_ids = train_data['User_id'].unique() 
    total_hits = 0
    total_precision = 0
    total_recall = 0
    total_users = len(user_ids)

    for user_id in user_ids:
        hits, precision, recall = calculate_top_k_hits_and_metrics(user_id, k, train_data, test_data)
        total_hits += hits
        total_precision += precision
        total_recall += recall

    avg_precision = total_precision / total_users if total_users > 0 else 0
    avg_recall = total_recall / total_users if total_users > 0 else 0

    if avg_precision + avg_recall > 0:
        f1_score = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
    else:
        f1_score = 0

    return total_hits, avg_precision, avg_recall, f1_score

k = 50
total_hits, avg_precision, avg_recall, f1_score = evaluate_f1_score(k, train_data, test_data)

print(f"Tổng số hits: {total_hits}")
print(f"Precision trung bình: {avg_precision:.4f}")
print(f"Recall trung bình: {avg_recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")


Không có sách nào để gợi ý cho người dùng ATDE9JYCPI0L1.


KeyboardInterrupt: 